# Day 4 Statistical Functions and Random Data
## Part 1: 기술 통계 함수
### 1.1 기본 통계량의 수학적 이해 

In [32]:
import numpy as np
rng = np.random.default_rng(42) # RNG : Random Number Generator, (seed)
data = rng.normal(loc = 100, scale = 15, size = 1000) # 정규분포에서 샘플을 뽑아, mean은 100, std은 15, 개수는 1000인 것.

In [33]:
print(data.mean())
print(data.std())

99.5666267350608
14.830835330947618


In [34]:
mean = np.mean(data)
median = np.median(data)
# 최빈값 사용 원하면 mode = stats.mode(data)

print(f"Mean: {mean:.4f}")
print(f"Median: {median:.4f}")

# 평균 - 중앙값 : skewness hint
print(f"Mean - Median = {mean - median:.4f}")


Mean: 99.5666
Median: 100.0927
Mean - Median = -0.5260


### 1.2 DDOF :  분산 / 표준편차

In [35]:
# ===== ===== ===== ===== ===== ===== ===== =====
# 산포도 (Dispersion)
# ===== ===== ===== ===== ===== ===== ===== =====

data = np.array([2, 4, 6, 8, 10])

# ddof : Delta Degrees of Freedom
# 모분산 : Population Variance : N으로 나눔
var_pop = np.var(data, ddof = 0)

# sample variance (불편추정량) : N-1로 나눔
# 표본분산은 평균을 추정하느라 자유도 1을 잃었기 때문에, N−1로 나눠야 모분산을 과소추정하지 않는다!
var_sample = np.var(data, ddof = 1)

print(f"모분산 (ddof=0): {var_pop:.4f}")
print(f"표본분산 (ddof=1): {var_sample:.4f}")


# 표준편차 = sqrt(분산)
std_pop = np.std(data, ddof=0)
std_sample = np.std(data, ddof=1)
print(f"모표준편차: {std_pop:.4f}")
print(f"표본표준편차: {std_sample:.4f}")


모분산 (ddof=0): 8.0000
표본분산 (ddof=1): 10.0000
모표준편차: 2.8284
표본표준편차: 3.1623


Q/A : 왜 ddof = 1을 써야 할까?

In [36]:
# test
np.random.seed(42)
population = np.random.normal(100, 15, size = 100000)
true_variance = np.var(population, ddof = 0)
print(f"모집단의 분산 : {true_variance:.2f}")

모집단의 분산 : 225.41


In [37]:
# 표본 크기 30으로 1000번 추출해보자.
n_experiments = 1000
sample_size = 30
var_ddof0 = []
var_ddof1 = []

rng = np.random.default_rng(42)
for _ in range(n_experiments):
    sample = rng.choice(population, size=sample_size, replace=False)
    var_ddof0.append(np.var(sample, ddof=0))
    var_ddof1.append(np.var(sample, ddof=1))
    
print(f"\nddof=0 평균 추정치: {np.mean(var_ddof0):.2f} (편향됨, 과소추정)")
print(f"ddof=1 평균 추정치: {np.mean(var_ddof1):.2f} (정확)")


ddof=0 평균 추정치: 217.68 (편향됨, 과소추정)
ddof=1 평균 추정치: 225.19 (정확)


### 1.3 분위수 / 백분위수
- Quantiles / Percentiles

1. 지수분포의 수학적 정의
- 지수분포(exponential distribution)의 확률밀도함수는 다음과 같다.

$$
f(x;\lambda) = \lambda e^{-\lambda x}, \quad x \ge 0
$$
그리고

$\lambda$ : rate parameter (단위 시간당 발생률)

---
2. 평균과 분산

- 지수분포의 이론적 성질은 다음과 같다.

$$
\mathbb{E}[X] = \frac{1}{\lambda}
$$

$$
\mathrm{Var}(X) = \frac{1}{\lambda^2}
$$

---
3. NumPy의 파라미터화 방식

NumPy는 $$\lambda$$ 대신 **scale**을 사용한다.

$$
\text{scale} = \frac{1}{\lambda}
$$

따라서

```python
rng.exponential(scale=10)
```

은 다음과 동등하다.

$$
\lambda = 0.1
$$

---

4. `scale`의 통계적 의미

지수분포에서는 다음 관계가 성립한다.

$$
\text{mean} = \text{std} = \text{scale}
$$

즉,

* 평균 = scale
* 표준편차 = scale

In [38]:
rng = np.random.default_rng(42)
data = rng.exponential(scale = 10, size = 1000)

In [39]:
p25 = np.percentile(data, 25)
p50 = np.percentile(data, 50)
p75 = np.percentile(data, 75)
p95 = np.percentile(data, 95)

print(f"25th percentila (Q1): {p25:.2f}")
print(f"50th percentila (Q2): {p50:.2f}")
print(f"75th percentila (Q3): {p75:.2f}")
print(f"95th percentila (Q4): {p95:.2f}")

25th percentila (Q1): 2.91
50th percentila (Q2): 6.80
75th percentila (Q3): 14.10
95th percentila (Q4): 30.88


In [40]:
# quantile의 범위를 0~1로 맞춘다면?
q1, q2, q3 = np.quantile(data, [0.25, 0.5, 0.75])
print(f"\nQuantiles: Q1={q1:.2f}, Q2={q2:.2f}, Q3={q3:.2f}")


Quantiles: Q1=2.91, Q2=6.80, Q3=14.10


In [41]:
# IQR (Interquartile Range) : 이상치 탐지
iqr = q3 - q1
lower_bound = q1-1.5*iqr
upper_bound = q3 + 1.5*iqr

outliers = data[(data < lower_bound) | (data > upper_bound)]

print(f"\nIQR: {iqr:.2f}")
print(f"이상치 범위: < {lower_bound:.2f} 또는 > {upper_bound:.2f}")
print(f"이상치 개수: {len(outliers)} ({100*len(outliers)/len(data):.1f}%)")


IQR: 11.19
이상치 범위: < -13.87 또는 > 30.88
이상치 개수: 50 (5.0%)


### 1.4 여러 분위 수 한 번에 계산해보기

In [42]:
percentiles = [4, 11, 23, 40, 60, 77, 89, 96]
values = np.percentile(data, percentiles)
print(values)

[ 0.4489727   1.08995916  2.57941637  4.97179247  9.52111088 14.8463589
 22.72879329 33.07953539]


In [43]:
for p, v in zip(percentiles, values):
    print(f"{p}th percentils : {v:8.2f}")

4th percentils :     0.45
11th percentils :     1.09
23th percentils :     2.58
40th percentils :     4.97
60th percentils :     9.52
77th percentils :    14.85
89th percentils :    22.73
96th percentils :    33.08


## Part 2 : axis parameter

### 2.1 Axis 직관적으로 이해하기
- 이 축을 따라 값들을 합친다!

In [44]:
arr = np.array([[1, 2, 3],
                [4, 5, 6]])
print(arr)
print(f"shape: {arr.shape}")

[[1 2 3]
 [4 5 6]]
shape: (2, 3)


In [45]:
# axis = 0 : 행으로 합쳐봅시다.
col_sum = np.sum(arr, axis=0)
print(f"axis = 0 : {col_sum}")
print(f"shape : {col_sum.shape}")

axis = 0 : [5 7 9]
shape : (3,)


In [46]:
# axis = 1 : 열로도 합쳐봅시다.
row_sum = np.sum(arr, axis = 1)
print(row_sum)
print(row_sum.shape)

[ 6 15]
(2,)


In [47]:
# 기본값
total = np.sum(arr)
total

np.int64(21)

### 2.2 3차원 배열

In [48]:
# rng = np.random.default_rng(42)
# rng.uniform : continuous uniform distribution에서 난수 생성. /
# X ~ u(15, 30)에서 모든 값이 동일 확률 발생한다는 뜻.
sst = rng.uniform(15, 30, size=(12, 5, 6)) # (12개월, 위도, 경도)
time_mean = np.mean(sst, axis = 0)
print(time_mean.shape)
global_mean = np.mean(sst)
print(f"{global_mean:.2f}")

(5, 6)
22.42


### 2.3 keepdims 로 차원 유지하기

In [49]:
data = np.array([[10, 20, 30],
                 [40, 50, 60],
                 [70, 80, 90]])

# 행별 평균 (with keepdims = False)
rowmean1 = np.mean(data, axis = 1)
print(rowmean1)
print(rowmean1.shape)

[20. 50. 80.]
(3,)


In [50]:
rowmean2 = np.mean(data, axis = 1, keepdims=True)
print(rowmean2)
print(f"{rowmean2.shape}")

[[20.]
 [50.]
 [80.]]
(3, 1)


In [51]:
deviations = data - rowmean2
print(deviations)

[[-10.   0.  10.]
 [-10.   0.  10.]
 [-10.   0.  10.]]


### 2.4 Z-score (x - mean) / std

In [52]:
# rng = np.random.default_rng(42)
# 온도 15-30 / 염분 33-37 / 클로로필 0.1-10

raw_data = np.column_stack([
    rng.uniform(15, 30, 100),
    rng.uniform(33, 37, 100),
    rng.uniform(0.1, 10, 100)
])
print(raw_data[:5])

[[23.02347229 36.42491602  2.0000195 ]
 [28.73617287 35.52426177  1.83707624]
 [23.5117217  34.39069454  8.13868319]
 [20.95849952 35.65011838  2.72064594]
 [21.67524082 35.68741675  9.14152332]]


In [53]:
raw_data.shape

(100, 3)

In [54]:
# 각 변수(col)별로 평균 / 표준편차 계산?
col_mean = np.mean(raw_data, axis=0, keepdims=True)
col_std = np.std(raw_data, axis=0, keepdims=True, ddof = 1)

print(f"평균 : {col_mean}")
print(f"표준편차 : {col_std}")

평균 : [[22.76646097 35.05572329  5.14635852]]
표준편차 : [[4.47373255 1.10606203 2.84431078]]


In [55]:
print(f"\n변수별 평균: {col_mean.flatten()}")
print(f"변수별 표준편차: {col_std.flatten()}")


변수별 평균: [22.76646097 35.05572329  5.14635852]
변수별 표준편차: [4.47373255 1.10606203 2.84431078]


In [56]:
z_score = (raw_data - col_mean) / col_std 
print(z_score[:5])

print(f"표준화 후 평균 : {np.mean(z_score, axis=0)}")
print(f"표준화 후 표준편차: {np.std(z_score, axis=0, ddof=1)}")

[[ 0.05744897  1.23789867 -1.1061868 ]
 [ 1.33439177  0.42360959 -1.16347423]
 [ 0.16658589 -0.6012581   1.05203858]
 [-0.40412819  0.5373976  -0.85282966]
 [-0.24391716  0.57111938  1.40461613]]
표준화 후 평균 : [-5.43454171e-16  3.66484620e-15 -1.54390389e-16]
표준화 후 표준편차: [1. 1. 1.]


## Part 3: 상관관계와 공분산
### 3.1 Correlation Coefficient

In [57]:
# ===== ===== ===== ===== ===== ===== ===== =====
# 피어슨 상관계수 : 두 변수 간 선형 관계 강도
# ===== ===== ===== ===== ===== ===== ===== =====

# rng = np.random.default_rng(42)
n = 100

# create data
x = rng.normal(0, 1, n)
noise = rng.normal (0, 0.1, n)

y_pos = 2 * x + noise
y_neg = -3 * x + noise
y_none = rng.normal(0, 1 ,n)

corr_matrix = np.corrcoef(x, y_pos)
print(corr_matrix)

[[1.         0.99863477]
 [0.99863477 1.        ]]


In [58]:
all_vars = np.vstack([x, y_pos, y_neg, y_none])
corr_all = np.corrcoef(all_vars)
print(np.around(corr_all, 3))

[[ 1.     0.999 -0.999  0.18 ]
 [ 0.999  1.    -0.996  0.182]
 [-0.999 -0.996  1.    -0.178]
 [ 0.18   0.182 -0.178  1.   ]]


### 3.2 Covariance
- Cov(X, Y) = E[(X-ux)(Y-uy)]

$$
\Sigma =
\begin{pmatrix}
\mathrm{Var}(x) & \mathrm{Cov}(x,y) \\
\mathrm{Cov}(y,x) & \mathrm{Var}(y)
\end{pmatrix}
$$

In [59]:
cov_matrix = np.cov(x, y_pos)
print(f" 공분산 행렬 :\n {cov_matrix}")

 공분산 행렬 :
 [[0.93662212 1.85936976]
 [1.85936976 3.70129538]]


## 대각 성분: 분산 (Variance)

첫 번째 대각 성분은 (x)의 분산이다.

$$
\mathrm{Var}(x) = 0.9401
$$

두 번째 대각 성분은 (y)의 분산이다.

$$
\mathrm{Var}(y) = 3.7961
$$

→ (y)가 (x)보다 변동성이 훨씬 크다.

---

## 비대각 성분: 공분산 (Covariance)

비대각 성분은 (x)와 (y)의 공분산이다.

$$
\mathrm{Cov}(x,y) = \mathrm{Cov}(y,x) = 1.8871
$$

이는

$$
\mathrm{Cov}(x,y) > 0
$$

이므로,
(x)가 증가할수록 (y)도 함께 증가하는 **양의 선형 관계**를 의미한다.

---

## 상관계수로 정규화한 해석

공분산은 단위 의존적이므로, 보통 상관계수를 함께 본다.

$$
\rho_{xy}
=========

\frac{\mathrm{Cov}(x,y)}
{\sqrt{\mathrm{Var}(x),\mathrm{Var}(y)}}
$$

수치를 대입하면,

$$
\rho_{xy}
\approx
\frac{1.887}
{\sqrt{0.940 \times 3.796}}
\approx 1
$$

→ 두 변수는 **거의 완전한 양의 상관관계**를 가진다.

---

## 기하학적 해석

이 공분산 행렬은 데이터 분포가

$$
\text{원형이 아닌, 대각선 방향으로 늘어진 타원}
$$

형태임을 의미한다.

→ PCA를 적용하면 첫 번째 주성분이 대부분의 분산을 설명한다.

---

## 요약

$$
\boxed{
\begin{aligned}
&\mathrm{Var}(y) \gg \mathrm{Var}(x) \
&\mathrm{Cov}(x,y) > 0 \
&x, y \text{는 거의 선형적으로 함께 변한다}
\end{aligned}
}
$$


In [60]:
r = cov_matrix[0, 1] / np.sqrt((cov_matrix[0, 0])*(cov_matrix[1, 1]))
print(r)

0.9986347713725302


### 3.3 해양 데이터 상관분석 (example)

In [61]:
# rng = np.random.default_rng(42)

# 가상 데이터 만들기
n_stations = 50

sst = rng.uniform(10, 28, n_stations) # SST

chl = 5-0.15*sst + rng.normal(0, 0.5, n_stations)
chl = np.maximum(chl, 0.01)

psu = 34 + 0.05*sst + rng.normal(0, 0.3, n_stations)

print("===== 해양 변수 간 상관분석 =====")
print(f"SST 범위 : {sst.min():.2f} ~ {sst.max():.2f} °C")
print(f"Chl 범위 : {chl.min():.2f} ~ {chl.max():.2f} ug/L")
print(f"sal 범위 : {psu.min():.2f} ~ {psu.max():.2f} psu")

===== 해양 변수 간 상관분석 =====
SST 범위 : 10.25 ~ 27.66 °C
Chl 범위 : 0.29 ~ 4.53 ug/L
sal 범위 : 33.94 ~ 35.85 psu


In [62]:
# 상관 행렬
vars_matrix = np.column_stack([sst, chl, psu])
corr = np.corrcoef(vars_matrix, rowvar = False)
labels = ['SST', 'CHL', 'SAL']
for i, label in enumerate(labels):
    print(f"{label:>3}  {corr[i, 0]:8.3f} {corr[i, 1]:8.3f} {corr[i, 2]:8.3f}")

SST     1.000   -0.859    0.678
CHL    -0.859    1.000   -0.491
SAL     0.678   -0.491    1.000


## Part 4 누적 연산과 차분
### 4.1 누적합과 누적곱

In [63]:
# 누적 연산
arr = np.array([1, 2, 3, 4, 5])

# cumulative sum
cumsum = np.cumsum(arr)
print(arr)
print(cumsum)

[1 2 3 4 5]
[ 1  3  6 10 15]


In [64]:
# cumulative product
cumprod = np.cumprod(arr)
print(cumprod)

[  1   2   6  24 120]


In [68]:
# 2D 배열에서 axis 지정 후 곱
arr2d = np.arange(1, 7).reshape(2, 3)
print(arr2d)

print(f"\n 행 방향으로의 누적 합 :\n {np.cumsum(arr2d, axis=0)}")
print(f"\n 열 방향으로의 누적 합 :\n {np.cumsum(arr2d, axis=1)}")

[[1 2 3]
 [4 5 6]]

 행 방향으로의 누적 합 :
 [[1 2 3]
 [5 7 9]]

 열 방향으로의 누적 합 :
 [[ 1  3  6]
 [ 4  9 15]]


### 4.2 실전: 누적 분포 함수 (CDF)

In [ ]:
### Empirical CDF (경험적 누적분포함수)

# rng = np.random.default_rng(42)
data = rng.normal(0, 1, 1000) # 평균이 0, 표준편차가 1인 데이터 1000개
sorted_data = np.sort(data)

# 각 데이터 포인트에 누적 확률(1/N,  2/N,  ...., N/N)
cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)

# Probability Example
value = 0
idx = np.searchsorted(sorted_data, value)  # X -> P : 값(0)이 몇 번째 위치에 들어가는지 찾아보기, 502번째라네.
print(idx)

prob = cdf[idx] if idx < len(cdf) else 1.0
print(f"P(X ≤ {value}) ≈ {prob:.3f}") 

# 백분위수 찾기 (역함수)
target_prob = 0.95
idx_95 = np.searchsorted(cdf, target_prob) # 확률을 먼저 찾음..?
print(idx_95)
value_95 = sorted_data[idx_95]

print(f"95th percentile ≈ {value_95:.3f}")

509
P(X ≤ 0) ≈ 0.510
949
95th percentile ≈ 1.636


### 4.3 차분 Difference
### np.diff

In [74]:
arr = np.array([1, 4, 9, 16, 25])

# 1st derivative approximation
diff1 = np.diff(arr)
print(arr)
print(diff1)

[ 1  4  9 16 25]
[3 5 7 9]


In [75]:
diff2 = np.diff(arr, n=2)
print(diff2)

[2 2 2]


In [76]:
sst_timeseries = np.array([15.2, 15.5, 16.1, 16.8, 17.2, 17.0, 16.5])
sst_change = np.diff(sst_timeseries)

print(f"\nSST 시계열: {sst_timeseries}")
print(f"SST 변화량: {sst_change}")
print(f"최대 상승: {np.max(sst_change):.1f}°C")
print(f"최대 하락: {np.min(sst_change):.1f}°C")


SST 시계열: [15.2 15.5 16.1 16.8 17.2 17.  16.5]
SST 변화량: [ 0.3  0.6  0.7  0.4 -0.2 -0.5]
최대 상승: 0.7°C
최대 하락: -0.5°C


In [77]:
# prepend / append 로 길이 유지
sst_change_full = np.diff(sst_timeseries, prepend = sst_timeseries[0])
print(f"길이 유지 차분: {sst_change_full}")

길이 유지 차분: [ 0.   0.3  0.6  0.7  0.4 -0.2 -0.5]


## Part 5 : NaN 처리
### 5.1 NaN이 있는 데이터 처리하기! 

In [ ]:
# ===== ===== ===== ===== ===== =====
data_with_nan = np.array([1.0, 2,0, np.nan, 4.0, 5.0, np.nan, 7.0])

print({data_with_nan}) # error : nan이 있으니까..
print(np.mean(data_with_nan)) # error : nan이 있으니까..
print(np.sum(data_with_nan))  # error : nan이 있으니까..

TypeError: unhashable type: 'numpy.ndarray'

In [79]:
print(np.nanmean(data_with_nan)) 
print(np.nansum(data_with_nan))
print(np.nanstd(data_with_nan))

3.1666666666666665
19.0
2.4094720491334933


In [80]:
print("\n=== NaN 처리 함수 목록 ===")
nan_funcs = ['nansum', 'nanmean', 'nanstd', 'nanvar', 
             'nanmin', 'nanmax', 'nanmedian', 'nanpercentile',
             'nanargmin', 'nanargmax', 'nancumsum', 'nancumprod']
for func in nan_funcs:
    print(f"  np.{func}()")


=== NaN 처리 함수 목록 ===
  np.nansum()
  np.nanmean()
  np.nanstd()
  np.nanvar()
  np.nanmin()
  np.nanmax()
  np.nanmedian()
  np.nanpercentile()
  np.nanargmin()
  np.nanargmax()
  np.nancumsum()
  np.nancumprod()


### 5.2 NaN 탐지, 처리

In [82]:
data = np.array([[1, 2, np.nan],
                 [4, np.nan, 6],
                 [7, 8, 9]])

print(f"data : \n {data}")

data : 
 [[ 1.  2. nan]
 [ 4. nan  6.]
 [ 7.  8.  9.]]


In [83]:
nan_mask = np.isnan(data)
print(nan_mask)

[[False False  True]
 [False  True False]
 [False False False]]


In [84]:
nan_count = np.sum(nan_mask)
print(f"NaN 개수 : {nan_count}")
print(f"유효 데이터 수 : {np.sum(~nan_mask)}")

NaN 개수 : 2
유효 데이터 수 : 7


In [90]:
# 행, 열별로 count 해볼까
count_rownan = np.sum(nan_mask, axis = 1, keepdims=True)
count_colnan = np.sum(nan_mask, axis = 0, keepdims=True)
print(f"행 NaN 개수 : \n {count_rownan}")
print(f"열 NaN 개수 : \n {count_colnan}")

행 NaN 개수 : 
 [[1]
 [1]
 [0]]
열 NaN 개수 : 
 [[0 1 1]]


In [91]:
# NaN 제거 후 1D 변환
valid_data_1D = data[~np.isnan(data)]
print(valid_data_1D)

[1. 2. 4. 6. 7. 8. 9.]


In [92]:
# NaN을 특정 값으로 대체
data_filled = np.where(np.isnan(data), 0, data) #0으로 바꿔주세요.
print(data_filled)

[[1. 2. 0.]
 [4. 0. 6.]
 [7. 8. 9.]]


In [93]:
# Q. NaN을 열 평균으로 대체
data_copy = data.copy()

for col in range(data_copy.shape[1]):
    col_data = data_copy[:, col]
    col_mean = np.nanmean(col_data)
    col_data[np.isnan(col_data)] = col_mean
print(data_copy)

[[1.  2.  7.5]
 [4.  5.  6. ]
 [7.  8.  9. ]]


## Part 6. 난수 생성 심화
### 6.1 새로운 Generator API vs Legacy API

`Generator의 장점`
- 더 나은 통계적 성질
- 스레드 안전성
- 전역 상태 오염 방지
- 직관적 method

In [94]:
# legacy method
np.random.seed(42)
old_style = np.random.random(5)
print(old_style)

[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]


In [95]:
rng = np.random.default_rng(42)
new_style = rng.random(5)
print(new_style)

[0.77395605 0.43887844 0.85859792 0.69736803 0.09417735]


In [96]:
print("\n=== Generator 주요 메서드 ===")
methods = [
    ('random()', '0~1 균등분포'),
    ('integers(low, high)', '정수 난수'),
    ('normal(loc, scale)', '정규분포'),
    ('uniform(low, high)', '균등분포'),
    ('choice(arr)', '배열에서 선택'),
    ('shuffle(arr)', '배열 섞기'),
    ('permutation(n)', '순열 생성'),
]
for method, desc in methods:
    print(f"  rng.{method:<25} # {desc}")


=== Generator 주요 메서드 ===
  rng.random()                  # 0~1 균등분포
  rng.integers(low, high)       # 정수 난수
  rng.normal(loc, scale)        # 정규분포
  rng.uniform(low, high)        # 균등분포
  rng.choice(arr)               # 배열에서 선택
  rng.shuffle(arr)              # 배열 섞기
  rng.permutation(n)            # 순열 생성


### 6.2 다양한 확률 분포

In [99]:
rng = np.random.default_rng(20)
n = 10000

In [100]:
# loc : location (평균)
# scale : 표준편차, 평균 대기 시간(발생률), 1회 발생 시 평균 대기 시간
# low / high : 최솟값 최댓값
# lam : 단위 시간당 평균 발생 횟수 $\lambda$
# n, p : 시행횟수, 성공 확률
# shape : 사건의 횟수 (ex. shape = 2라면, 사건이 2번 발생할 때까지 기다리겠다는 뜻)
# a, b : (성공 가중치, 실패 가중치)
# mean / sigma : 로그를 취한 값의 평균, 로그를 취한 값의 표준편차
# 

# 1. Normal / Gaussian Distribution
normal = rng.normal(loc = 0, scale = 1, size = n) 

# 2. 균등분포 Uniform
uniform = rng.uniform(low=0, high=1, size=n)

# 3. 지수분포 Exponential
exponential = rng.exponential(scale=2.0, size=n)

# 4. 포아송 분포 Poisson
poisson = rng.poisson(lam=5, size = n)

# 5. 이항분포 Binomial
binomial = rng.binomial(n=10, p=0.5, size=n)

# 6. 감마분포 Gamma
gamma = rng.gamma(shape=2, scale=2, size=n)

# 7. 베타분포 beta
beta = rng.beta(a=2, b=5, size=n)

# 8. 로그정규분포 lognormal
lognormal = rng.lognormal(mean=0, sigma=1, size=n)

### 6.3 샘플링 셔플링

In [101]:
rng = np.random.default_rng(42)
population = np.array(['Lee', 'Kim', 'Cheon', 'Park', 'Choi'])

In [103]:
# 복원 추출
with_replacement = rng.choice(population, size=10, replace=True)
print(with_replacement)

['Cheon' 'Choi' 'Park' 'Park' 'Park' 'Park' 'Cheon' 'Lee' 'Choi' 'Cheon']


In [104]:
# 비복원 추출
without_replacement = rng.choice(population, size=3, replace=False)
print(without_replacement)

['Kim' 'Park' 'Lee']


In [105]:
# 가중치 부여 샘플
weights = [0.5, 0.2, 0.15, 0.1, 0.05]
weighted = rng.choice(population, size=1000, p=weights)

print(f"\n 가중치 샘플링: ")
for item in population:
    count = np.sum(weighted == item)
    print(f"  {item}: {count} ({100*count/len(weighted):.1f}%)")


 가중치 샘플링: 
  Lee: 506 (50.6%)
  Kim: 194 (19.4%)
  Cheon: 151 (15.1%)
  Park: 100 (10.0%)
  Choi: 49 (4.9%)


---